**Sentiment Analysis using LSTM Network or GRU.**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load and preprocess data
data = pd.read_csv(r"C:\Users\ganga\Downloads\IMDB Dataset.csv")
data['review'] = data['review'].str.replace('<.*?>', '').str.replace('[^A-Za-z]', ' ')
data['review'] = data['review'].str.lower()
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

x_train, x_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2)

# Tokenize and pad
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train = pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=100)
x_test = pad_sequences(tokenizer.texts_to_sequences(x_test), maxlen=100)

# Build model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, 32, input_length=100),
    LSTM(64),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(x_train, y_train, epochs=3, batch_size=128)

# Evaluate
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Prediction Function
def predict_sentiment(review):
    # Preprocess input
    review = review.lower()
    review = ''.join([c if c.isalpha() or c.isspace() else ' ' for c in review])
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=100)

    # Predict
    result = model.predict(padded_sequence)[0][0]
    sentiment = 'positive' if result >= 0.5 else 'negative'
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}")

# Example Usage
while True:
    user_review = input("Enter a movie review (or type 'exit' to quit): ")
    if user_review.lower() == 'exit':
        break
    predict_sentiment(user_review)
